# MODÉLISATION MATHÉMATIQUE DU SYSTÈME DE FEUX DE CIRCULATION

**Auteur :** Khaoula  

---

## Table des Matières

1. [Introduction et Description du Système](#1-introduction-et-description-du-système)
2. [Classification du Système](#2-classification-du-système)
3. [Modélisation des Arrivées par Loi de Poisson](#3-modélisation-des-arrivées-par-loi-de-poisson)
4. [Modélisation par Automate Fini](#4-modélisation-par-automate-fini)
5. [Modélisation par Chaîne de Markov](#5-modélisation-par-chaîne-de-markov)
6. [Modélisation des Files d'Attente M/M/1](#6-modélisation-des-files-dattente-mm1)
7. [Analyse de Stabilité du Système](#7-analyse-de-stabilité-du-système)
8. [Application aux Scénarios](#8-application-aux-scénarios)
9. [Modèle avec Durées Aléatoires](#9-modèle-avec-durées-aléatoires)
10. [Validation Mathématique](#10-validation-mathématique)

---

## 1. Introduction et Description du Système

### 1.1 Contexte du Projet

Notre projet consiste à modéliser et simuler un **carrefour à feux de circulation** où deux voies se croisent perpendiculairement :

**Schéma du carrefour :**

```
                    Voie B (Nord → Sud)
                           ↓
                           |
                    [Feu piéton B]
                           |
                       [Feu B]
                           |
    ←───────[Feu A]────────┼────────[Feu A]─────────→
         Voie A (Ouest → Est)
                           |
                       [Feu B]
                           |
                    [Feu piéton A]
                           |
                           ↓
```

### 1.2 Composants du Système

#### Entités (Clients)
- **Véhicules** : arrivant sur la voie A et la voie B
- **Piétons** : souhaitant traverser pendant la phase dédiée

#### Serveurs (Ressources)
- **Voie A** : permet le passage des véhicules lorsque le feu est vert
- **Voie B** : permet le passage des véhicules lorsque le feu est vert
- **Passages piétons** : actifs pendant la phase piétons

#### États des Feux

| État | Feu Voie A | Feu Voie B | Feux Piétons | Durée |
|------|------------|------------|--------------|-------|
| S1 | 🟢 Vert | 🔴 Rouge | 🔴 Rouge | T_A = 30s |
| S2 | 🟡 Jaune | 🔴 Rouge | 🔴 Rouge | T_jaune = 3s |
| S3 | 🔴 Rouge | 🟢 Vert | 🔴 Rouge | T_B = 25s |
| S4 | 🔴 Rouge | 🟡 Jaune | 🔴 Rouge | T_jaune = 3s |
| S5 | 🔴 Rouge | 🔴 Rouge | 🟢 Vert | T_piétons = 15s |

**Durée totale d'un cycle :**
```
T_cycle = T_A + T_jaune + T_B + T_jaune + T_piétons
T_cycle = 30 + 3 + 25 + 3 + 15 = 76 secondes
```

### 1.3 Variables du Système

#### Paramètres d'Entrée (Contrôlables)

| Variable | Notation | Valeur | Unité | Description |
|----------|----------|--------|-------|-------------|
| Durée feu vert A | T_A | 30 | s | Temps d'ouverture voie A |
| Durée feu vert B | T_B | 25 | s | Temps d'ouverture voie B |
| Durée feu jaune | T_jaune | 3 | s | Phase de transition |
| Durée phase piétons | T_piétons | 15 | s | Traversée piétons |
| Taux arrivée A | λ_A | 0.5 | véh/s | Flux voie A |
| Taux arrivée B | λ_B | 0.5 | véh/s | Flux voie B |

#### Variables d'État

- `N_A(t)` = nombre de véhicules en attente sur voie A à l'instant t
- `N_B(t)` = nombre de véhicules en attente sur voie B à l'instant t
- `État_feu_A(t)` ∈ {Vert, Jaune, Rouge}
- `État_feu_B(t)` ∈ {Vert, Jaune, Rouge}
- `Phase_actuelle` ∈ {S1, S2, S3, S4, S5}

#### Indicateurs de Performance

| Indicateur | Notation | Unité | Objectif |
|------------|----------|-------|----------|
| Temps moyen d'attente A | W_A | secondes | Minimiser |
| Temps moyen d'attente B | W_B | secondes | Minimiser |
| Longueur moyenne file A | L_A | véhicules | Minimiser |
| Longueur moyenne file B | L_B | véhicules | Minimiser |
| Taux d'utilisation A | ρ_A | sans unité | < 1 (stabilité) |
| Taux d'utilisation B | ρ_B | sans unité | < 1 (stabilité) |

---

## 2. Classification du Système

D'après le **Chapitre I** du cours (Types de systèmes), notre système peut être classifié :

### 2.1 Selon la Nature Temporelle

**→ Système DISCRET**

**Justification :** Les changements d'état se produisent à des instants précis :
- Changement de couleur des feux
- Arrivée d'un véhicule
- Départ d'un véhicule

Le temps entre deux événements n'affecte pas l'état du système.

### 2.2 Selon le Déterminisme

**→ Système STOCHASTIQUE**

**Justification :** L'arrivée des véhicules est **aléatoire** et suit un processus de Poisson. Même si les durées des feux sont déterministes, le comportement global intègre de l'aléatoire.

### 2.3 Selon l'Évolution

**→ Système DYNAMIQUE**

**Justification :** L'état du système évolue dans le temps :
- Les files d'attente augmentent ou diminuent
- Les feux changent de couleur de manière cyclique

### 2.4 Conclusion

Notre système est un **système dynamique discret stochastique**.

---

## 3. Modélisation des Arrivées par Loi de Poisson

### 3.1 Choix de la Loi de Poisson

Pour modéliser les **arrivées de véhicules**, nous utilisons un **processus de Poisson** de paramètre λ.

#### Propriétés Justifiant ce Choix

| Propriété | Application aux Véhicules |
|-----------|---------------------------|
| **Indépendance** | L'arrivée d'un véhicule n'influence pas l'arrivée du suivant |
| **Aléatoire** | Les instants d'arrivée ne sont pas prédictibles |
| **Taux constant** | Sur une période (ex: heure de pointe), le taux moyen reste stable |
| **Événements rares** | Dans un petit intervalle de temps, peu de véhicules arrivent |

### 3.2 Définition Mathématique

Le nombre de véhicules N(t) arrivant dans un intervalle de temps [0, t] suit une **loi de Poisson** :

```
P(N(t) = k) = (λt)^k × e^(-λt) / k!

Où :
- k = nombre de véhicules (0, 1, 2, 3, ...)
- λ = taux moyen d'arrivée (véhicules/seconde)
- t = durée de l'intervalle (secondes)
- e = 2.71828... (constante d'Euler)
```

**Caractéristiques :**
```
Espérance : E[N(t)] = λt
Variance : Var(N(t)) = λt
```

### 3.3 Lien avec la Loi Exponentielle

**Théorème fondamental :**

> Si les arrivées suivent un processus de Poisson(λ), alors le **temps entre deux arrivées successives** suit une **loi Exponentielle(λ)**.

```
Temps inter-arrivée T ~ Exp(λ)

Densité : f(t) = λ × e^(-λt)

Espérance : E[T] = 1/λ
```

**Application à notre projet :**

```
Voie A : λ_A = 0.5 véhicules/seconde
→ Temps moyen entre deux voitures = 1/0.5 = 2 secondes

Voie B : λ_B = 0.5 véhicules/seconde
→ Temps moyen entre deux voitures = 1/0.5 = 2 secondes
```

### 3.4 Calculs pour Notre Système

#### Exemple : Probabilités d'Arrivée en 10 Secondes

Pour la voie A avec λ_A = 0.5 véh/s :

**Nombre moyen de véhicules en 10 secondes :**
```
E[N(10)] = λ_A × 10 = 0.5 × 10 = 5 véhicules
```

**Probabilités :**

| k | P(N = k) | Interprétation |
|---|----------|----------------|
| 0 | 0.0067 | 0.67% - Très rare |
| 3 | 0.1404 | 14.04% |
| 5 | 0.1755 | 17.55% - Plus probable |
| 7 | 0.1044 | 10.44% |
| 10 | 0.0181 | 1.81% - Rare |

**Calcul détaillé pour k = 5 :**
```
P(N(10) = 5) = (5)^5 × e^(-5) / 5!
              = 3125 × 0.0067 / 120
              = 0.1755 = 17.55%
```

### 3.5 Propriété Sans Mémoire

La loi exponentielle possède la **propriété sans mémoire** :

```
P(T > s + t | T > s) = P(T > t)
```

**Interprétation pour notre projet :**

Si aucun véhicule n'est arrivé depuis 5 secondes, la probabilité qu'un véhicule arrive dans les 2 prochaines secondes est **la même** que si on venait de commencer à observer.

Cette propriété est appelée **propriété de Markov** et justifie l'utilisation des chaînes de Markov (voir section 5).

---

## 4. Modélisation par Automate Fini

### 4.1 Définition Formelle

D'après le **Chapitre I - Outils de modélisation**, un automate fini est défini par :

**A = (E, Σ, δ, e₀, F)**

Où :
- **E** : ensemble des états
- **Σ** : alphabet (événements)
- **δ : E × Σ → E** : fonction de transition
- **e₀** : état initial
- **F** : états finaux

### 4.2 Application au Système de Feux

#### Ensemble des États (E)

```
E = {S1, S2, S3, S4, S5}
```

| État | Configuration des Feux | Durée |
|------|------------------------|-------|
| S1 | A=Vert, B=Rouge, Piétons=Rouge | 30s |
| S2 | A=Jaune, B=Rouge, Piétons=Rouge | 3s |
| S3 | A=Rouge, B=Vert, Piétons=Rouge | 25s |
| S4 | A=Rouge, B=Jaune, Piétons=Rouge | 3s |
| S5 | A=Rouge, B=Rouge, Piétons=Vert | 15s |

#### Alphabet (Σ)

```
Σ = {T_fin}
```

Où `T_fin` = événement "la durée de l'état courant est écoulée"

#### Fonction de Transition (δ)

```
δ(S1, T_fin) = S2    (après 30 secondes)
δ(S2, T_fin) = S3    (après 3 secondes)
δ(S3, T_fin) = S4    (après 25 secondes)
δ(S4, T_fin) = S5    (après 3 secondes)
δ(S5, T_fin) = S1    (après 15 secondes)
```

#### État Initial

```
e₀ = S1  (le système démarre avec la voie A en vert)
```

### 4.3 Diagramme d'États

```
        T_A=30s      T_jaune=3s     T_B=25s       T_jaune=3s
    S1 ────────→ S2 ────────────→ S3 ──────────→ S4 ───────────→ S5
    ↑                                                               │
    │                                                               │
    └───────────────────────────────────────────────────────────────┘
                          T_piétons=15s
```

### 4.4 Propriétés de l'Automate

✅ **Déterministe** : Une seule transition possible depuis chaque état  
✅ **Complet** : Toute combinaison (état, événement) a une transition définie  
✅ **Cyclique** : Le système revient toujours à l'état initial S1  

**Période du cycle :**
```
T_cycle = 30 + 3 + 25 + 3 + 15 = 76 secondes
```

---

## 5. Modélisation par Chaîne de Markov

### 5.1 Chaîne de Markov à Temps Discret

D'après le **Chapitre II** du cours, une chaîne de Markov possède la **propriété de Markov** :

> "Le futur ne dépend que du présent, pas du passé"

Notre système de feux peut être modélisé comme une chaîne de Markov où les transitions se font à intervalles réguliers (fin de chaque phase).

### 5.2 Matrice de Transition P

La matrice P décrit les probabilités de passer d'un état i à un état j :

```
P_{ij} = P(X_{n+1} = j | X_n = i)
```

**Matrice pour notre système :**

```
          S1    S2    S3    S4    S5
      ┌                              ┐
  S1  │  0     1     0     0     0  │
  S2  │  0     0     1     0     0  │
  S3  │  0     0     0     1     0  │
  S4  │  0     0     0     0     1  │
  S5  │  1     0     0     0     0  │
      └                              ┘
```

**Interprétation :**
- De S1, on va **toujours** vers S2 (probabilité = 1)
- De S2, on va **toujours** vers S3 (probabilité = 1)
- C'est une chaîne de Markov **déterministe**

### 5.3 Propriétés de la Chaîne

#### Irréductibilité

La chaîne est **irréductible** car tous les états communiquent :

```
S1 → S2 → S3 → S4 → S5 → S1
```

Tous les états sont accessibles depuis n'importe quel état.

#### Périodicité

La chaîne est **périodique** de période **d = 5** :

```
Pour l'état S1 :
T(S1) = {n : P^n(S1, S1) > 0} = {5, 10, 15, 20, ...}
d = pgcd(T(S1)) = 5
```

On revient à chaque état tous les 5 pas exactement.

#### Distribution Stationnaire

La distribution stationnaire π satisfait :

```
π = π × P
Σ π_i = 1
```

Pour notre chaîne cyclique uniforme :

```
π = [0.2, 0.2, 0.2, 0.2, 0.2]
```

**Interprétation :** À long terme, le système passe **20% du temps** dans chaque état.

**Vérification :**
```
π × P = [0.2, 0.2, 0.2, 0.2, 0.2] × P
      = [0.2, 0.2, 0.2, 0.2, 0.2] ✓
```

### 5.4 Proportions de Temps

En tenant compte des durées réelles :

```
Proportion temps en S1 = T_A / T_cycle = 30/76 = 39.5%
Proportion temps en S2 = T_jaune / T_cycle = 3/76 = 3.9%
Proportion temps en S3 = T_B / T_cycle = 25/76 = 32.9%
Proportion temps en S4 = T_jaune / T_cycle = 3/76 = 3.9%
Proportion temps en S5 = T_piétons / T_cycle = 15/76 = 19.7%

Total = 100% ✓
```

**Conclusion importante :**
- Voie A est verte **39.5%** du temps
- Voie B est verte **32.9%** du temps
- Phase piétons = **19.7%** du temps

---

## 6. Modélisation des Files d'Attente M/M/1

### 6.1 Notation de Kendall

D'après le **Chapitre III** du cours, chaque voie est modélisée comme une **file d'attente M/M/1** :

**M/M/1/∞/FIFO**

- **M** (1ère position) : Arrivées selon processus de Poisson
- **M** (2ème position) : Service selon loi exponentielle
- **1** : Un seul serveur (une voie)
- **∞** : Capacité de la file illimitée
- **FIFO** : Premier arrivé, premier servi

### 6.2 Paramètres du Modèle

#### Taux d'Arrivée (λ)

```
λ_A = 0.5 véhicules/seconde (voie A)
λ_B = 0.5 véhicules/seconde (voie B)
```

**Temps inter-arrivée moyen :**
```
E[T_inter] = 1/λ = 1/0.5 = 2 secondes
```

#### Taux de Service (μ)

Le taux de service dépend du cycle des feux.

**Hypothèse :** Quand le feu est vert, un véhicule franchit le carrefour instantanément.

**Calcul du taux de service effectif :**

Le feu de la voie A n'est vert que pendant T_A secondes par cycle.

```
Proportion de temps vert : α_A = T_A / T_cycle = 30/76 ≈ 0.395
```

Si pendant le vert, le débit est μ_max, alors :

```
μ_A = μ_max × α_A
```

**Exemple avec μ_max = 1 véh/s :**
```
μ_A = 1 × 0.395 = 0.395 véhicules/seconde
μ_B = 1 × 0.329 = 0.329 véhicules/seconde
```

### 6.3 Facteur d'Utilisation (ρ)

Le facteur d'utilisation mesure la charge du système :

```
ρ = λ / μ
```

**Pour notre système :**

```
ρ_A = λ_A / μ_A = 0.5 / 0.395 ≈ 1.27 ❌
ρ_B = λ_B / μ_B = 0.5 / 0.329 ≈ 1.52 ❌
```

### 6.4 ⚠️ Problème Critique : ρ > 1

**Condition de stabilité (Chapitre III) :**

```
Système stable ⟺ ρ < 1 ⟺ λ < μ
```

Avec ρ > 1, les deux voies sont **surchargées** et les files d'attente croissent indéfiniment ! ❌

### 6.5 Formules des Indicateurs (si ρ < 1)

Lorsque ρ < 1, les formules suivantes s'appliquent :

#### Nombre Moyen de Véhicules dans le Système
```
L = ρ / (1 - ρ)
```

#### Nombre Moyen en Attente
```
L_q = ρ² / (1 - ρ)
```

#### Temps Moyen dans le Système
```
W = L / λ = 1 / (μ - λ)
```

#### Temps Moyen d'Attente
```
W_q = L_q / λ = ρ / (μ - λ)
```

**⚠️ Ces formules ne sont valables que si ρ < 1 !**

---

## 7. Analyse de Stabilité du Système

### 7.1 Diagnostic du Problème

Avec les paramètres initiaux :

```
λ_A = λ_B = 0.5 véh/s
μ_A = 0.395 véh/s
μ_B = 0.329 véh/s

→ ρ_A = 1.27 > 1 (INSTABLE)
→ ρ_B = 1.52 > 1 (INSTABLE)
```

Le système est **instable** : les files d'attente croissent indéfiniment.

### 7.2 Solutions Possibles

#### Solution 1 : Réduire le Taux d'Arrivée

Pour rendre le système stable :

```
Pour ρ_A < 1 : λ_A < μ_A
λ_A < 0.395 véhicules/seconde

Pour ρ_B < 1 : λ_B < μ_B
λ_B < 0.329 véhicules/seconde
```

**Nouveau paramétrage stable :**
```
λ_A = λ_B = 0.3 véhicules/seconde

Vérification :
ρ_A = 0.3 / 0.395 = 0.76 < 1 ✓
ρ_B = 0.3 / 0.329 = 0.91 < 1 ✓
```

#### Solution 2 : Augmenter les Durées de Vert

```
Pour μ_A > λ_A avec λ_A = 0.5 :
μ_A = μ_max × (T_A / T_cycle) > 0.5
T_A / T_cycle > 0.5

Si μ_max = 1 et T_cycle = 76s :
T_A > 38 secondes (au lieu de 30s)
```

#### Solution 3 : Modèle M/M/c (Plusieurs Voies)

Augmenter le nombre de serveurs (voies en parallèle) - voir Chapitre III, modèle M/M/S.

### 7.3 Paramètres Retenus pour l'Analyse

Pour la suite de l'analyse, nous utilisons :

```
λ_A = λ_B = 0.3 véhicules/seconde
μ_A = 0.395 véhicules/seconde
μ_B = 0.329 véhicules/seconde

→ ρ_A = 0.76 ✓
→ ρ_B = 0.91 ✓
```

---

## 8. Application aux Scénarios

### 8.1 Scénario 1 : Trafic Léger (Configuration Actuelle)

#### Paramètres
```
λ_A = 0.3 véh/s
λ_B = 0.3 véh/s
T_A = 30s
T_B = 25s
T_cycle = 76s
```

#### Calculs pour Voie A

**1. Taux d'utilisation :**
```
μ_A = 1 × (30/76) = 0.395 véh/s
ρ_A = 0.3 / 0.395 = 0.76 = 76% ✓
```

**2. Longueur moyenne de file :**
```
L_A = ρ_A / (1 - ρ_A) = 0.76 / (1 - 0.76) = 3.17 véhicules
```

**3. Longueur moyenne en attente :**
```
L_q_A = ρ_A² / (1 - ρ_A) = 0.76² / 0.24 = 2.41 véhicules
```

**4. Temps moyen dans le système :**
```
W_A = L_A / λ_A = 3.17 / 0.3 = 10.57 secondes
```

**5. Temps moyen d'attente :**
```
W_q_A = L_q_A / λ_A = 2.41 / 0.3 = 8.03 secondes
```

#### Calculs pour Voie B

**1. Taux d'utilisation :**
```
μ_B = 1 × (25/76) = 0.329 véh/s
ρ_B = 0.3 / 0.329 = 0.91 = 91% ⚠️
```

**2. Longueur moyenne de file :**
```
L_B = 0.91 / (1 - 0.91) = 10.11 véhicules
```

**3. Temps moyen d'attente :**
```
W_q_B = 0.91 / (0.329 - 0.3) = 30.67 secondes
```

#### Analyse

```
Voie A :
✓ Charge acceptable (76%)
✓ Temps d'attente raisonnable (8s)
✓ File courte (2-3 véhicules)

Voie B :
⚠️ Charge élevée (91%)
⚠️ Temps d'attente long (31s)
⚠️ File longue (9-10 véhicules)

→ La voie B est proche de la saturation !
→ Il faut augmenter T_B ou réduire λ_B
```

### 8.2 Scénario 2 : Trafic Asymétrique

#### Paramètres
```
λ_A = λ_B = 0.4 véh/s (augmentation)
T_A = 40s (augmentation)
T_B = 20s (réduction)
T_cycle = 81s
```

#### Calculs

**Voie A :**
```
μ_A = 1 × (40/81) = 0.494 véh/s
ρ_A = 0.4 / 0.494 = 0.81 = 81% ✓

L_A = 0.81 / 0.19 = 4.26 véhicules
W_q_A = 4.26 / 0.4 = 10.65 secondes
```

**Voie B :**
```
μ_B = 1 × (20/81) = 0.247 véh/s
ρ_B = 0.4 / 0.247 = 1.62 > 1 ❌

→ SYSTÈME INSTABLE pour la voie B !
```

#### Analyse

```
❌ Configuration NON VIABLE
❌ Voie B saturée (ρ > 1)
→ Il faut réduire λ_B ou augmenter T_B
```

### 8.3 Scénario 3 : Configuration Optimisée

#### Objectif
Équilibrer les deux voies pour avoir ρ_A ≈ ρ_B ≈ 0.80

#### Calculs

Si on veut ρ_A = ρ_B = 0.80 avec λ_A = λ_B = 0.3 :

```
ρ = λ / μ = 0.80
μ = λ / 0.80 = 0.3 / 0.80 = 0.375 véh/s
```

Pour les deux voies :
```
μ_A = μ_B = 0.375 véh/s
```

Avec μ = μ_max × (T_vert / T_cycle) :
```
T_A / T_cycle = 0.375
T_B / T_cycle = 0.375
```

Si T_cycle = 76s :
```
T_A = 0.375 × 76 = 28.5 secondes
T_B = 0.375 × 76 = 28.5 secondes
```

**Configuration équilibrée :**
```
T_A = T_B = 28s
T_jaune = 3s
T_piétons = 14s
T_cycle = 28 + 3 + 28 + 3 + 14 = 76s
```

#### Résultats

```
ρ_A = ρ_B = 0.80 = 80% ✓

L_A = L_B = 0.80 / 0.20 = 4 véhicules
W_q_A = W_q_B = 4 / 0.3 = 13.33 secondes

→ Configuration ÉQUILIBRÉE
→ Performance acceptable sur les deux voies
```

### 8.4 Tableau Comparatif des Scénarios

| Scénario | λ (véh/s) | T_A (s) | T_B (s) | ρ_A | ρ_B | W_q_A (s) | W_q_B (s) | État |
|----------|-----------|---------|---------|-----|-----|-----------|-----------|------|
| 1. Actuel | 0.3 | 30 | 25 | 0.76 | 0.91 | 8.0 | 30.7 | ⚠️ B surchargé |
| 2. Asymétrique | 0.4 | 40 | 20 | 0.81 | 1.62 | 10.7 | ∞ | ❌ B instable |
| 3. Optimisé | 0.3 | 28 | 28 | 0.80 | 0.80 | 13.3 | 13.3 | ✅ Équilibré |

---

## 9. Modèle avec Durées Aléatoires

### 9.1 Chaîne de Markov à Temps Continu

Pour modéliser des **durées aléatoires**, nous utilisons une **chaîne de Markov à temps continu** (CMTC) du **Chapitre II**.

Dans une CMTC, le temps de séjour dans chaque état suit une **loi exponentielle**.

### 9.2 Matrice Génératrice Q

**Définition :**
```
Q = [q_ij]

où q_ij = taux de transition de i vers j (i ≠ j)
    q_ii = -Σ(j≠i) q_ij
```

**Pour notre système :**

Si les durées suivent des lois exponentielles :
```
Durée en S1 ~ Exp(λ₁) où λ₁ = 1/T_A = 1/30
Durée en S2 ~ Exp(λ₂) où λ₂ = 1/T_jaune = 1/3
Durée en S3 ~ Exp(λ₃) où λ₃ = 1/T_B = 1/25
Durée en S4 ~ Exp(λ₄) où λ₄ = 1/3
Durée en S5 ~ Exp(λ₅) où λ₅ = 1/T_piétons = 1/15
```

**Matrice Q :**

```
      S1       S2       S3       S4       S5
  ┌                                          ┐
S1│ -0.033   0.033     0        0        0  │
S2│  0      -0.333   0.333      0        0  │
S3│  0        0      -0.040   0.040      0  │
S4│  0        0        0      -0.333   0.333│
S5│ 0.067     0        0        0     -0.067│
  └                                          ┘
```

**Propriétés :**
- Chaque ligne somme à 0 ✓
- Les éléments diagonaux sont négatifs ✓
- Les transitions hors diagonale sont les taux λᵢ

### 9.3 Distribution Uniforme (Alternative)

Une approche plus simple consiste à utiliser des **lois uniformes** :

```
T_A ~ Uniforme[25, 35] secondes (moyenne = 30, variance = ±5)
T_B ~ Uniforme[20, 30] secondes (moyenne = 25, variance = ±5)
```

**Génération en simulation :**
```
T_A = 25 + random() × 10
T_B = 20 + random() × 10
```

**Avantages :**
- Plus simple à implémenter
- Durées bornées (réaliste)
- Moyenne et variance contrôlées

**Désavantages :**
- Moins de modélisation théorique
- Pas de propriété de Markov

### 9.4 Impact sur les Performances

Avec des durées aléatoires, les indicateurs deviennent des **variables aléatoires** :

```
E[W_q] = temps d'attente moyen espéré
Var(W_q) = variance du temps d'attente
```

**Conséquences :**
- Les files fluctuent plus
- Risque de dépassements ponctuels
- Nécessité de simulation Monte Carlo pour estimation

---

## 10. Validation Mathématique

### 10.1 Vérification de Cohérence

#### Loi de Little

**Théorème de Little (universel) :**
```
L = λ × W
```

**Vérification pour la voie A (Scénario 1) :**
```
L_A = 3.17 véhicules
λ_A = 0.3 véh/s
W_A = 10.57 s

Vérification : L_A = λ_A × W_A
               3.17 = 0.3 × 10.57
               3.17 ≈ 3.17 ✓
```

#### Conservation de la Probabilité

**Matrice de transition P :**
```
Σⱼ P_ij = 1 pour tout i

Vérification :
Ligne S1 : 0 + 1 + 0 + 0 + 0 = 1 ✓
Ligne S2 : 0 + 0 + 1 + 0 + 0 = 1 ✓
```

#### Distribution Stationnaire

**Équation : π × P = π**
```
π = [0.2, 0.2, 0.2, 0.2, 0.2]

Vérification :
π × P = [0.2×0 + 0.2×0 + ... , 0.2×1 + ..., ...]
      = [0.2, 0.2, 0.2, 0.2, 0.2] ✓
```

### 10.2 Tests de Cohérence du Modèle

#### Test 1 : Stabilité

```
Condition : ρ < 1 pour toutes les voies

Scénario 1 :
ρ_A = 0.76 < 1 ✓
ρ_B = 0.91 < 1 ✓

→ Système STABLE
```

#### Test 2 : Conservation du Flux

```
Nombre arrivées = Nombre départs (en régime permanent)

λ_A × T_simulation ≈ Nombre véhicules servis voie A

Exemple : T = 1000s
Arrivées théoriques = 0.3 × 1000 = 300 véhicules
Départs attendus ≈ 300 véhicules (si système stable)
```

#### Test 3 : Temps de Cycle

```
T_cycle théorique = 76s

Vérification en simulation :
Mesurer le temps entre deux passages en S1
→ Doit être ≈ 76s en moyenne
```

### 10.3 Méthode de Validation par Simulation

**Approche Monte Carlo :**

1. **Générer N simulations** (N = 1000)
2. **Calculer les statistiques empiriques**
3. **Comparer avec les valeurs théoriques**

**Exemple pour W_q_A :**

```
Théorie : W_q_A = 8.03 secondes

Simulation :
- Moyenne empirique : W̄_q_A
- Écart-type empirique : σ̂

Test d'acceptation (intervalle de confiance 95%) :
W̄_q_A ∈ [W_q_A - 1.96σ̂/√N, W_q_A + 1.96σ̂/√N]

Si W̄_q_A ∈ [7.5, 8.5] → Modèle validé ✓
```

---

## 11. Synthèse et Récapitulatif

### 11.1 Outils Mathématiques Utilisés

| Outil | Application | Objectif |
|-------|-------------|----------|
| **Loi de Poisson** | Arrivées de véhicules | Modéliser le flux aléatoire |
| **Loi Exponentielle** | Temps inter-arrivée | Lien avec Poisson |
| **Automate Fini** | Contrôle des feux | Gestion des transitions |
| **Chaîne de Markov Discrète** | Séquence des états | Analyse cyclique |
| **Chaîne de Markov Continue** | Durées aléatoires | Extension stochastique |
| **File M/M/1** | Service des véhicules | Calcul des performances |

### 11.2 Équations Clés

#### Cycle Déterministe
```
T_cycle = T_A + 2×T_jaune + T_B + T_piétons = 76s
```

#### Proportions de Temps Vert
```
α_A = T_A / T_cycle = 30/76 ≈ 0.395
α_B = T_B / T_cycle = 25/76 ≈ 0.329
```

#### Taux de Service Effectif
```
μ_A = μ_max × α_A
μ_B = μ_max × α_B
```

#### Condition de Stabilité
```
ρ = λ/μ < 1
```

#### Indicateurs de Performance (si ρ < 1)
```
L = ρ/(1-ρ)           Nombre moyen de véhicules
W = 1/(μ-λ)           Temps moyen dans le système
L_q = ρ²/(1-ρ)        Nombre moyen en attente
W_q = ρ/(μ-λ)         Temps moyen d'attente
L = λ × W             Loi de Little
```

### 11.3 Résultats Principaux

**Configuration Actuelle (30s/25s) :**
```
✓ Voie A stable (ρ = 76%)
⚠️ Voie B proche saturation (ρ = 91%)
→ Temps d'attente asymétrique (8s vs 31s)
```

**Configuration Optimisée (28s/28s) :**
```
✓ Équilibrage parfait (ρ = 80% partout)
✓ Temps d'attente identiques (13s)
→ Meilleure équité entre usagers
```

### 11.4 Recommandations

1. **Pour un trafic léger (λ = 0.3 véh/s) :**
   - Utiliser T_A = T_B = 28s
   - Garantit ρ < 0.85 sur toutes les voies

2. **Pour un trafic moyen (λ = 0.4 véh/s) :**
   - Augmenter T_A et T_B à 35s chacun
   - Réduire T_piétons si nécessaire

3. **Pour un trafic intense (λ > 0.5 véh/s) :**
   - Passer à un modèle M/M/c (plusieurs voies)
   - Ou implémenter des feux adaptatifs

---

## 12. Conclusion

### 12.1 Objectifs Atteints

✅ Classification du système (discret, stochastique, dynamique)  
✅ Modélisation des arrivées par processus de Poisson  
✅ Formalisation par automate fini et chaînes de Markov  
✅ Application de la théorie des files M/M/1  
✅ Analyse de stabilité et calcul des indicateurs  
✅ Proposition de configurations optimisées  

### 12.2 Contributions au Projet

Cette modélisation mathématique fournit :

1. **Cadre théorique rigoureux** pour la simulation (Sarah)
2. **Prédictions analytiques** à comparer avec les résultats (Tasnim)
3. **Critères d'optimisation** (minimiser W_q, maintenir ρ < 0.85)

### 12.3 Limites et Extensions

**Limites du modèle :**
- Temps de traversée considéré nul
- Pas de virages (flux linéaire)
- Capacité de file infinie

**Extensions possibles :**
- Modèle M/G/1 (service non-exponentiel)
- Files finies M/M/1/K
- Priorités entre voies
- Feux adaptatifs (fonction de N_A, N_B)

---

## Références

1. **Cours de Modélisation et Simulation** - Pr. Mohammed Nadjib KOUAHLA
   - Chapitre I : Types de systèmes et outils de modélisation
   - Chapitre II : Chaînes de Markov
   - Chapitre III : Files d'attente

2. Gross, D., & Harris, C. M. (1998). *Fundamentals of Queueing Theory*

3. Ross, S. M. (2014). *Introduction to Probability Models*

4. Kleinrock, L. (1975). *Queueing Systems Volume I: Theory*

---

**Document rédigé par : Khaoula**  
**Dans le cadre du module : Modélisation et Simulation**  
**Université 08 Mai 1945 Guelma - Département d'Informatique**  
**Décembre 2024**